In [ ]:
Пример организации вызова функции моделью:

In [3]:
from __future__ import annotations

from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

import dotenv, os

import time

dotenv.load_dotenv()

config = dotenv.dotenv_values()

def calculator(expression: str) -> str:
    print(f'ответ калькулятора {expression=}')
    return "-5"


def weather(location: str, date: str) -> str:
    print(f"функция погоды вернула {location=} и {date=}")
    return "-12 градусов тепла"

def local_time() -> str:
    ans = time.strftime('%d.%m.%Y г. %H:%M:%S', time.localtime())
    return ans

def process_tool_calls(tool_calls) -> dict[str, list[dict]]:
    """
    Пример способа организации обработки вызова функций в общем случае
    """

    function_map = {
        'calculator': calculator,
        'Weather': weather,
        'LocalTime': local_time
    }

    result = []
    for tool_call in tool_calls:
        # пока что доступны только функции
        assert tool_call.function

        function = function_map[tool_call.function.name]

        answer = function(**tool_call.function.arguments)  # type: ignore[operator]

        result.append({'name': tool_call.function.name, 'content': answer})

    return {'tool_results': result}


def main() -> None:
    sdk = YCloudML(
        folder_id=config["YC_FOLDER_ID"],
        auth=IAMTokenAuth(config["YC_IAM_TOKEN"]),
        enable_server_data_logging=False,
    )

    calculator_tool = sdk.tools.function(
        name="calculator",
        description="Простой калькулятор для выполнения арифметических операций и @ операций.",
        # параметры могут содержать корректную jsonschema с параметрами функции и описанием
        parameters={
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "Математическое выражение для вычисления (e.g., '2 + 3 * 4').",
                }
            },
            "required": ["expression"],
        }
    )

    

    from pydantic import BaseModel, Field

    class Weather(BaseModel):
        """Получение погоды в указанное время в указанном месте"""

        # Очень важно описать все аргументы на естественном языке
        location: str = Field(description="Название локации, для которой ищется погода")
        date: str = Field(description="Дата, в которой заинтересован пользователь")

    # также можно передать в качестве параметров модель pydantic; в таком случае jsonschema будет извлечена из переданной модели
    weather_tool = sdk.tools.function(Weather)

    class LocalTime(BaseModel):
        """Получение локального времени на компьютере"""
        pass

    time_tool = sdk.tools.function(LocalTime)

    model = sdk.models.completions('yandexgpt')

    # tools must be bound to model object via .configure method and would be used in all
    # model calls from this model object.
    model = model.configure(tools=[calculator_tool, weather_tool, time_tool], temperature=0)

    for question in ["Сколько будет 5@-1?", "Какая погода была в Москве 12 Мая 2025 года?", "Какое сейчас локальное время?"]:
        # необходимо тщательно и аккуратно сохранять контекст для передачи результатов функций для модели после вызова функции 
        messages: list = [
            {"role": "system", "text": "Используй русский язык в ответах"},
            question
        ]

        result = model.run(messages)

        # if result object contains tool calls, it would be treated as a tool call history next time
        # you will pass "messages" to a model.
        # if there is no tool calls, it still will be treated as a usual text message with a proper role.
        # NB: result is a shortcut for result[0].alternative in this context.
        messages.append(result)

        if result.tool_calls:
            tool_results = process_tool_calls(result.tool_calls)
            # we adding a special tool results message to a history
            messages.append(tool_results)
            # and running model second time to pass tool results with a context back to model.
            result = model.run(messages)
        else:
            print(result.text)
            print('в данном примере модель должна вызывать функцию каждый раз;')

        print(f"Ответ модели на вопрос {question}:", result.text)


if __name__ == '__main__':
    main()

ответ калькулятора expression='5@-1'
Ответ модели на вопрос Сколько будет 5@-1?: Результат выражения 5@-1 равен -5.
функция погоды вернула location='Москва' и date='12 мая 2025 года'
Ответ модели на вопрос Какая погода была в Москве 12 Мая 2025 года?: 12 мая 2025 года в Москве было -12 градусов тепла.
Ответ модели на вопрос Какое сейчас локальное время?: Сейчас локальное время: 16:19:32.
